In [1496]:
import os

proj_dir = '/Users/nickbachelder/Desktop/Kaggle/Linemen'
os.chdir( os.path.join(proj_dir, 'code/objects') )

%run load_modules.ipynb
%run tracking_objects.ipynb

In [1268]:
data_dir = os.path.join(proj_dir, 'data' )
os.chdir( proj_dir )
os.system("rm -rf {}".format( data_dir ))
os.system('mkdir data')
os.chdir( data_dir )
os.system("kaggle competitions download -c nfl-big-data-bowl-2023")
os.system( 'unzip {}.zip'.format( os.path.join(data_dir, 'nfl-big-data-bowl-2023')) )

curr_cd = os.getcwd()
files = [x for x in os.listdir(curr_cd) if os.path.isfile(os.path.join(curr_cd, x))]

player_files = [os.path.join(curr_cd, x) for x in files if x.endswith('players.csv')]
player_tracking_files = [os.path.join(curr_cd, x) for x in files if x.startswith('week')]
play_details_files = [os.path.join(curr_cd, x) for x in files if x.endswith('ScoutingData.csv')]
play_info_files = [os.path.join(curr_cd, x) for x in files if x.endswith('plays.csv')]


100%|██████████| 172M/172M [00:23<00:00, 7.63MB/s] 



Archive:  /Users/nickbachelder/Desktop/Kaggle/Linemen/data/nfl-big-data-bowl-2023.zip
  inflating: games.csv               
  inflating: pffScoutingData.csv     
  inflating: players.csv             
  inflating: plays.csv               
  inflating: week1.csv               
  inflating: week2.csv               
  inflating: week3.csv               
  inflating: week4.csv               
  inflating: week5.csv               
  inflating: week6.csv               
  inflating: week7.csv               
  inflating: week8.csv               


In [1269]:
all_data = master_track(track_paths = player_tracking_files, 
                            play_info_path = play_info_files[0],
                            play_details_path = play_details_files[0],
                            players_path = player_files[0])

In [1270]:
week = 6
all_data.load_training_data(week = week)

Getting week 1 training data


100%|██████████| 1170/1170 [06:13<00:00,  3.13it/s] 


Getting week 2 training data


100%|██████████| 1062/1062 [05:22<00:00,  3.29it/s]


Getting week 3 training data


...

Getting week 4 training data


100%|██████████| 1105/1105 [05:33<00:00,  3.31it/s]


Getting week 5 training data


100%|██████████| 1104/1104 [06:13<00:00,  2.95it/s]


Done


In [1271]:
all_data.get_averages_up_to_week(week = week)

Getting week 1 average metrics by position and blocker number
Getting week 2 average metrics by position and blocker number
Getting week 3 average metrics by position and blocker number
Getting week 4 average metrics by position and blocker number
Getting week 5 average metrics by position and blocker number
Done


In [1272]:
# figure out where these issues are coming from later and fix

all_dat_labels = all_data.get_rush_sequences_labels(week = week, normalize = True)

training_dat, test_dat = train_test_split(all_dat_labels, test_size = 0.2)

# remove nas (fix these nas being created later)
bad_indexs = []
print('Train NA')
for value in range(len(training_dat)):
    any_na = training_dat[value][0].isnull().values.any()
    if any_na == True:
        bad_indexs.append(value)
        print(value)

for index in sorted(bad_indexs, reverse=True):
    del training_dat[index]


bad_indexs = []
print('Test NA')
for value in range(len(test_dat)):
    any_na = test_dat[value][0].isnull().values.any()
    if any_na == True:
        bad_indexs.append(value)
        print(value)

for index in sorted(bad_indexs, reverse=True):
    del test_dat[index]

print(f'Length train : {len(training_dat)}')
print(f'Length test : {len(test_dat)}')

Train NA
Test NA
Length train : 4463
Length test : 1116


In [1397]:
# model building

model = SequenceModel(n_features = training_dat[0][0].shape[1], n_classes = 2, n_hidden = 15, n_layers = 1)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= 0.01)
train_dataset = RushDataset(sequences = training_dat, sequence_length=50)
train_loader = DataLoader(train_dataset, batch_size = 50, shuffle = True)
test_dataset = RushDataset(sequences = test_dat, sequence_length=30)
test_loader = DataLoader(test_dataset, batch_size = 50, shuffle = True)

lstm_model = train_rush_lstm(train_loader, test_loader, model, loss_function, optimizer, num_epochs = 20)

Epoch 0
---------
Overall Train loss: 0.6318761457999548 , Overall Train AUC: 0.6437469227315579
Train loss for period 1: 0.6719293223487006 , Train AUC for period 1: 0.5331990331478167
Train loss for period 2: 0.679140274061097 , Train AUC for period 2: 0.5398721381051726
Train loss for period 3: 0.6724392526679569 , Train AUC for period 3: 0.5514438655667849
Train loss for period 4: 0.6632153428263134 , Train AUC for period 4: 0.5751122032043927
Train loss for period 5: 0.6461465623643663 , Train AUC for period 5: 0.6197564663249683
Train loss for period 6: 0.625798664159245 , Train AUC for period 6: 0.6680703248949087
Train loss for period 7: 0.6050287322865592 , Train AUC for period 7: 0.7159584376357744
Train loss for period 8: 0.5901237398386001 , Train AUC for period 8: 0.7519237399391049
Train loss for period 9: 0.5832782573170132 , Train AUC for period 9: 0.7702012328260727
Train loss for period 10: 0.5816612508561876 , Train AUC for period 10: 0.7755176492308375
Overall test 

In [1295]:
export_w1_for_plotting = all_data.training_data_team.get('1')
export_w1_for_plotting.to_csv("w1_for_plotting")

In [1474]:
all_data.play_details_df.query('(disrupt_individual == True)').playId.unique()[1:50]

array([ 2021090900349,  2021090900687,  2021090900788,  2021090900925,
       20210909001506, 20210909001587, 20210909002211, 20210909002233,
       20210909002279, 20210909002298, 20210909002330, 20210909002419,
       20210909002554, 20210909002735, 20210909002780, 20210909002915,
       20210909003014, 20210909003134, 20210909003306, 20210909003364,
       20210909003655, 20210909003724, 20210909003746, 20210909003962,
       20210909004189, 20210909004211, 20210909004287, 20210909004393,
       20210909004538, 20210909004560, 20210909004624,  2021091200176,
        2021091200231,  2021091200348,  2021091200705,  2021091200843,
        2021091200864,  2021091200966, 20210912001085, 20210912001153,
       20210912001271, 20210912001326, 20210912001459, 20210912002063,
       20210912002145, 20210912002275, 20210912002399, 20210912002631,
       20210912003480])

In [1499]:
folder = os.path.join(proj_dir, 'data' )

# Sack Plots

download_play_plot_df(master_track = all_data, play_id = 20210912082010, model = lstm_model, folder = folder)
download_metric_plot_df(master_track = all_data, play_id = 20210912082010, model = lstm_model, folder = folder)

# Hit Plots

download_play_plot_df(master_track = all_data, play_id = 20210912003480, model = lstm_model, folder = folder)
download_metric_plot_df(master_track = all_data, play_id = 20210912003480, model = lstm_model, folder = folder)

# Hurry Plots

download_play_plot_df(master_track = all_data, play_id = 2021090900687, model = lstm_model, folder = folder)
download_metric_plot_df(master_track = all_data, play_id = 2021090900687, model = lstm_model, folder = folder)

# Non-Disrupt Plots

download_play_plot_df(master_track = all_data, play_id = 2021090900282, model = lstm_model, folder = folder)
download_metric_plot_df(master_track = all_data, play_id = 2021090900282, model = lstm_model, folder = folder)


# Prob difference example plot

download_play_prob_differences(master_track = all_data, play_id = 20210909003962, model = lstm_model, folder = folder)

In [1281]:
ranking_dict = get_player_metrics(week)
ranking_df = get_top_player_metrics(master_track = all_data, metric_dict = ranking_dict, min_play_n = 10)
ranking_df[0:50].to_csv('final_metric_rankings.csv')

0it [00:00, ?it/s]
100%|██████████| 1138/1138 [07:54<00:00,  2.40it/s]


,nflId,metric,n_play,height,weight,birthDate,collegeName,officialPosition,displayName,rank
0,44976,0.089196,16,6-0,223,1994-07-28,Boston College,OLB,Matt Milano,1
1,35522,0.078344,16,6-3,335,1988-06-08,Kentucky,DT,Corey Peters,2
2,43341,0.072961,13,6-1,227,1994-11-04,Louisiana State,ILB,Deion Jones,3
3,53587,0.067375,13,6-3,305,NaN,NaN,DT,Daviyon Nixon,4
4,43641,0.065984,12,5-9,184,1994-03-09,Mississippi,CB,Mike Hilton,5
5,43378,0.064154,59,6-2,305,1993-02-07,South Carolina State,DT,Javon Hargrave,6
6,52475,0.063784,11,6-5,241,1997-12-27,Notre Dame,OLB,Julian Okwara,7
7,43325,0.060362,12,6-1,244,1995-09-03,UCLA,MLB,Myles Jack,8
8,43638,0.059585,48,6-3,292,1992-10-23,Ferris State,DT,Justin Zimmer,9
9,47971,0.057539,11,5-11,224,1996-10-12,West Virginia,ILB,David Long,10


In [1394]:
all_data.

,nflId,height,weight,birthDate,collegeName,officialPosition,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,28963,6-5,240,1982-03-02,"Miami, O.",QB,Ben Roethlisberger
2,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
3,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
4,30078,6-2,228,1982-11-24,Harvard,QB,Ryan Fitzpatrick
...,...,...,...,...,...,...,...
1674,53991,6-1,320,NaN,NaN,DT,Forrest Merrill
1675,53994,6-5,300,NaN,NaN,C,Ryan McCollum
1676,53999,6-4,312,NaN,NaN,DT,Jack Heflin
1677,54006,6-6,330,NaN,NaN,T,Jake Curhan


In [1103]:
all_data.search_track_weeks(variables = ['playId'], variable_values = [202110030076])

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,week,snap_frame,time_after_snap
0,2021100300,202110030076,33084.0,1,2021-10-03T17:03:50.200,2.0,ATL,left,84.55,23.69,0.02,0.02,0.01,286.55,271.13,None,6,6.0,-0.5
1,2021100300,202110030076,33084.0,2,2021-10-03T17:03:50.300,2.0,ATL,left,84.54,23.69,0.02,0.02,0.00,286.55,270.26,None,6,6.0,-0.4
2,2021100300,202110030076,33084.0,3,2021-10-03T17:03:50.400,2.0,ATL,left,84.54,23.68,0.02,0.02,0.01,286.55,268.60,None,6,6.0,-0.3
3,2021100300,202110030076,33084.0,4,2021-10-03T17:03:50.500,2.0,ATL,left,84.54,23.68,0.01,0.02,0.00,286.55,268.17,None,6,6.0,-0.2
4,2021100300,202110030076,33084.0,5,2021-10-03T17:03:50.600,2.0,ATL,left,84.54,23.68,0.01,0.02,0.00,286.55,267.79,None,6,6.0,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2021100300,202110030076,NaN,25,2021-10-03T17:03:52.600,NaN,football,left,85.79,23.62,0.55,1.11,0.11,NaN,NaN,None,6,6.0,1.9
663,2021100300,202110030076,NaN,26,2021-10-03T17:03:52.700,NaN,football,left,82.29,20.56,22.11,2.24,4.65,NaN,NaN,None,6,6.0,2.0
664,2021100300,202110030076,NaN,27,2021-10-03T17:03:52.800,NaN,football,left,80.86,19.27,21.02,12.21,1.92,NaN,NaN,None,6,6.0,2.1
665,2021100300,202110030076,NaN,28,2021-10-03T17:03:52.900,NaN,football,left,79.34,17.94,19.13,17.80,2.02,NaN,NaN,None,6,6.0,2.2


[202110030076,
 202110030159,
 202110030297,
 202110030476,
 202110030683,
 202110030878,
 202110030976,
 202110031097,
 202110031155,
 202110031255,
 202110031356,
 202110031380,
 202110040081,
 2021093000169,
 2021093000209,
 2021093000276,
 2021093000329,
 2021093000351,
 2021093000449,
 2021093000549,
 2021093000599,
 2021093000621,
 2021093000766,
 2021093000788,
 2021093000893,
 2021093000957,
 2021093000981,
 2021100300142,
 2021100300208,
 2021100300253,
 2021100300293,
 2021100300317,
 2021100300367,
 2021100300389,
 2021100300473,
 2021100300495,
 2021100300566,
 2021100300667,
 2021100300864,
 2021100300913,
 2021100300959,
 2021100301198,
 2021100301292,
 2021100301449,
 2021100301471,
 2021100301518,
 2021100301698,
 2021100301909,
 2021100301957,
 2021100301981,
 2021100302142,
 2021100302208,
 2021100302401,
 2021100302467,
 2021100302580,
 2021100302853,
 2021100302972,
 2021100303111,
 2021100303173,
 2021100303199,
 2021100303274,
 2021100303370,
 2021100303463,
 2021

In [468]:
play_dets = all_data.play_details_df
play_dets['disrupt'] = play_dets.pff_hit + play_dets.pff_hurry + play_dets.pff_sack
print( f'Percent of player play disruptions: {play_dets.disrupt.sum() / len(play_dets.disrupt)}')
team_dets = play_dets.groupby('playId').agg({'disrupt' : [sum]})
team_dets.columns = ["_".join(x) for x in np.array(team_dets.columns).ravel()]
team_dets = team_dets.reset_index()
print( f'Percent of team play disruptions: {len(team_dets.loc[team_dets.disrupt_sum > 0]) / len(team_dets)}')

Percent of player play disruptions: 0.022958343514613237
Percent of team play disruptions: 0.37583265163024426


In [ ]:
# sack / hurry / hit are dependent on both pass coverage and lineman and we only care about linemen. 
# Instead of using sack / hit /hurry, use P(Sack, Hurry, Hit @ time t) given conditions for team
# to evaluate players, need some sort of metric for 'contribution to P(Sack, Hurry, Hit @ time t) on play'
# to do so try to do  or sommin    [ P(Sack, Hurry, Hit @ time t replacing player with rusher average) / P(Sack, Hurry, Hit @ time t) ] - P(Sack, Hurry, Hit @ time t)




# new idea: since there is no team level time of sack
# find model that we can track each player prob of hit/hurry/sack 
# each player route will have a prob of sack at time t. 
# to compare use average route stats in similar situation: note may have to import more data (ie. play type etc)


# also: need to create qb_metric for 'facing rusher' 

In [ ]:
import matplotlib.pyplot as plt
plot_overall_avgs = all_dist_dfs.query("time_after_snap > 0").groupby(by = ['time_after_snap']).mean().reset_index()

plt.title = "Average Rusher Distance From QB by Seconds After Snap"
plt.xlabel = "Seconds After Snap"
plt.plot('time_after_snap',  'rusher_distance_from_qb', data = plot_overall_avgs, linestyle='dashed')
plt.ylabel = "Rusher Distance From QB"
plt.show()

In [ ]:
### want to get groups of players with similar starting positions: conlusion: just use number of blockers on play for now, maybe add in weigt of blocker later

# this_week_starting_pos = all_data.training_data_individual.get('1')
# this_week_starting_pos = this_week_training_individual.loc[this_week_training_individual.time_after_snap == 0, ['playId', 'nflId', 'rusher_distance_from_qb', 'y']]



# gmm_train = this_week_starting_pos.drop(['playId', 'nflId'], axis = 1)
# gmm = mixture.GaussianMixture(n_components=2).fit(gmm_train)
# gmm_labels = gmm.predict(gmm_train).astype(str)

# this_week_starting_pos['starting_pos_class'] = gmm_labels
# this_week_starting_pos.groupby('starting_pos_class').groups.items()
# plot_df = this_week_starting_pos.drop(['playId', 'nflId'], axis = 1).reset_index(drop = 1)
# plot_df.starting_pos_class.unique()
# plot_df

# groups = plot_df.groupby('starting_pos_class')
# # Plot
# fig, ax = plt.subplots()
# ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
# for name, group in groups:
#     ax.plot(group.rusher_distance_from_qb, group.y, marker='o', linestyle='', ms=12, label=name)
# ax.legend()

# plt.show()

# ret_df = this_week_starting_pos.loc[:, ['playId', 'nflId', 'starting_pos_class']]
# ret_df